In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

load sample data

In [6]:
input_bam = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'
input_maf = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs_TN_wxs_bam/runs/C3L-00677_2a9edf1e-76df-4595-9125-fb9b9d919621/cromwell-workdir/cromwell-executions/pecgs_TN_wxs_bam.cwl/8e1e5c0e-f904-4995-9f97-2cfad0bbd9fc/call-run_tindaisy/tindaisy2.cwl/5d22279a-37e0-4f73-b58d-61624aca4f6e/call-vcf2maf/execution/result.maf'

bed = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources_backup/neoscan/refseq_hg38_june29/proteome.bed'
f_allele = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources_backup/neoscan/netMHC-4.0/Linux_x86_64/data/allelelist'
netmhc = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources_backup/neoscan/netMHC-4.0/netMHC'
ref_dir = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources_backup/neoscan/refseq_hg38_june29'


make test input yaml

In [7]:
template = yaml.safe_load(open('/home/estorrs/pecgs-pipeline/submodules/pecgs-neoscan/cwl/template.neoscan.yaml'))
template


{'bam': {'class': 'File', 'path': 'a/file/path'},
 'bed': {'class': 'File', 'path': 'a/file/path'},
 'f_allele': {'class': 'File', 'path': 'a/file/path'},
 'input_type': 'a_string',
 'maf': {'class': 'File', 'path': 'a/file/path'},
 'netmhc': {'class': 'File', 'path': 'a/file/path'},
 'ref_dir': {'class': 'Directory', 'path': 'a/dir/path'}}

In [9]:
template['bam']['path'] = input_bam
template['maf']['path'] = input_maf

template['bed']['path'] = bed
template['f_allele']['path'] = f_allele
template['netmhc']['path'] = netmhc
template['ref_dir']['path'] = ref_dir

In [10]:
template

{'bam': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/test_samples/C3L-00677/wxs/1d6c4f0f-09c4-4472-a19e-30ee7e0ecf64_gdc_realn.bam'},
 'bed': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources_backup/neoscan/refseq_hg38_june29/proteome.bed'},
 'f_allele': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources_backup/neoscan/netMHC-4.0/Linux_x86_64/data/allelelist'},
 'input_type': 'a_string',
 'maf': {'class': 'File',
  'path': '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/pecgs_TN_wxs_bam/runs/C3L-00677_2a9edf1e-76df-4595-9125-fb9b9d919621/cromwell-workdir/cromwell-executions/pecgs_TN_wxs_bam.cwl/8e1e5c0e-f904-4995-9f97-2cfad0bbd9fc/call-run_tindaisy/tindaisy2.cwl/5d22279a-37e0-4f73-b58d-61624aca4f6e/call-vcf2maf/execution/result.maf'},
 'netmhc': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources_backup/neoscan/

In [12]:
yaml.safe_dump(template, open('../tests/data/neoscan/C3L-00677_inputs.yaml', 'w'))

make cromwell compute1 config

In [13]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [14]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677'
output_fp = '../tests/data/neoscan/C3L-00677.cromwell-config-db.compute1.dat'
args['queue'] = 'dinglab'
bsub.save_compute1_cromwell_template(workflow_root, output_fp, queue=args['queue'])

make execution script

In [15]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/neoscan'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'C3L-00677.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-neoscan', 'cwl', 'neoscan.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'C3L-00677_inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active', '/scratch1/fs1/dinglab/estorrs']

In [16]:
start_commands, run_command = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes,
                                                     workflow_root=workflow_root)
filepath = os.path.join('../tests/data/neoscan/start_server.sh')
bsub.write_command_file(start_commands, filepath)
filepath = os.path.join('../tests/data/neoscan/run_job.sh')
bsub.write_command_file([run_command], filepath)

In [17]:
for c in start_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/neoscan:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/neoscan /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/neoscan/C3L-00677 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab/estorrs:/scratch1/fs1/dinglab/estorrs"
bgadd -L 10 /estorrs/test_cromwell
bsub -n 1 -q dinglab-interactive -G compute-dinglab -a 'docker(mwyczalkowski/cromwell-runner)' -g /estorrs/test_cromwell -J 0a751cfa-9903-4f3c-866c-

In [18]:
print(run_command)

/usr/bin/java -Dconfig.file=/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/neoscan/C3L-00677.cromwell-config-db.compute1.dat -Djavax.net.ssl.trustStorePassword=changeit -Djavax.net.ssl.trustStore=/gscmnt/gc2560/core/genome/cromwell/cromwell.truststore -jar /usr/local/cromwell/cromwell-47.jar run -t cwl -i /storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/neoscan/C3L-00677_inputs.yaml /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline/submodules/pecgs-neoscan/cwl/neoscan.cwl
